In [14]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Load dataset
df = pd.read_csv(r"C:\Users\Lulay\Desktop\Kuliah\Semester 4\Dasar Kecerdasan Artifisial\Tugas\W10_Tugas_Besar\engine_data.csv")

In [15]:
# Helper function to get fuzzy range
def get_ranges(column):
    q25 = df[column].quantile(0.25)
    q50 = df[column].quantile(0.5)
    q75 = df[column].quantile(0.75)
    min_val = df[column].min()
    max_val = df[column].max()
    return min_val, q25, q50, q75, max_val

# Get ranges
rpm_min, rpm_q25, rpm_q50, rpm_q75, rpm_max = get_ranges('Engine rpm')
lop_min, lop_q25, lop_q50, lop_q75, lop_max = get_ranges('Lub oil pressure')
fp_min, fp_q25, fp_q50, fp_q75, fp_max = get_ranges('Fuel pressure')

# Define fuzzy input variables
engine_rpm = ctrl.Antecedent(np.arange(rpm_min, rpm_max+1, 1), 'engine_rpm')
lub_oil_pressure = ctrl.Antecedent(np.arange(lop_min, lop_max+0.1, 0.1), 'lub_oil_pressure')
fuel_pressure = ctrl.Antecedent(np.arange(fp_min, fp_max+0.1, 0.1), 'fuel_pressure')

# Define fuzzy output variable (binary: 0 = bad, 1 = good)
engine_condition = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'engine_condition')

# Membership functions for inputs
engine_rpm['low'] = fuzz.trimf(engine_rpm.universe, [rpm_min, rpm_min, rpm_q50])
engine_rpm['medium'] = fuzz.trimf(engine_rpm.universe, [rpm_q25, rpm_q50, rpm_q75])
engine_rpm['high'] = fuzz.trimf(engine_rpm.universe, [rpm_q50, rpm_max, rpm_max])

lub_oil_pressure['low'] = fuzz.trimf(lub_oil_pressure.universe, [lop_min, lop_min, lop_q50])
lub_oil_pressure['normal'] = fuzz.trimf(lub_oil_pressure.universe, [lop_q25, lop_q50, lop_q75])
lub_oil_pressure['high'] = fuzz.trimf(lub_oil_pressure.universe, [lop_q50, lop_max, lop_max])

fuel_pressure['low'] = fuzz.trimf(fuel_pressure.universe, [fp_min, fp_min, fp_q50])
fuel_pressure['normal'] = fuzz.trimf(fuel_pressure.universe, [fp_q25, fp_q50, fp_q75])
fuel_pressure['high'] = fuzz.trimf(fuel_pressure.universe, [fp_q50, fp_max, fp_max])

# Membership functions for output
engine_condition['bad'] = fuzz.trimf(engine_condition.universe, [0, 0, 0.5])
engine_condition['good'] = fuzz.trimf(engine_condition.universe, [0.5, 1, 1])

# Define fuzzy rules
rule1 = ctrl.Rule(engine_rpm['low'] & lub_oil_pressure['low'] & fuel_pressure['low'], engine_condition['bad'])
rule2 = ctrl.Rule(engine_rpm['medium'] & lub_oil_pressure['normal'] & fuel_pressure['normal'], engine_condition['bad'])
rule3 = ctrl.Rule(engine_rpm['high'] & lub_oil_pressure['high'] & fuel_pressure['high'], engine_condition['good'])

# Create control system
engine_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
engine_simulation = ctrl.ControlSystemSimulation(engine_ctrl)

# Predict for a single row
row = df.iloc[0]
engine_simulation.input['engine_rpm'] = row['Engine rpm']
engine_simulation.input['lub_oil_pressure'] = row['Lub oil pressure']
engine_simulation.input['fuel_pressure'] = row['Fuel pressure']
engine_simulation.compute()